In [1]:
2+7

9

In [5]:
import pandas as pd
import numpy as np
import urllib.parse
from sqlalchemy import create_engine, text
import os

# 1. DATABASE CONFIGURATION
USER = "root"
PASSWORD = urllib.parse.quote_plus("Elan@2004") 
HOST = "localhost"
DATABASE = "amazon_project"

engine = create_engine(f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}/{DATABASE}")

# 2. LOAD DATA
cleaned_file = "../data/amazon_india_cleaned_final.csv"
if not os.path.exists(cleaned_file):
    cleaned_file = r"C:\Users\ELANSURYA.K\3D Objects\amazon project\data\amazon_india_cleaned_final.csv"

print("📥 Reading data...")
df = pd.read_csv(cleaned_file, low_memory=False)

# 3. TRANSFORMATION & CLEANING
print("🧹 Cleaning and Preparing tables...")

# Handle Dates
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df = df.dropna(subset=['order_date'])

# --- FIX: Handling NULLs and Boolean Conversion for MySQL ---
# Replace actual NaNs/NULLs with 0
df['is_prime_member'] = df['is_prime_member'].fillna(0)
df['is_festival_sale'] = df['is_festival_sale'].fillna(0)

# Map string 'True'/'False' to 1/0 and ensure they are integers
for col in ['is_prime_member', 'is_festival_sale']:
    df[col] = df[col].astype(str).str.upper().map({'TRUE': 1, 'FALSE': 0, '1': 1, '0': 0, '1.0': 1, '0.0': 0})
    df[col] = df[col].fillna(0).astype(int)

# Products Table
products = df[['product_id', 'product_name', 'category', 'brand']].drop_duplicates(subset=['product_id'])

# Customers Table
customers = df[['customer_id', 'customer_city', 'customer_state', 'is_prime_member']].drop_duplicates(subset=['customer_id'])

# Time Dimension
time_dim = pd.DataFrame()
time_dim['date_key'] = df['order_date'].dt.date
time_dim = time_dim.drop_duplicates()
time_dim['year'] = pd.to_datetime(time_dim['date_key']).dt.year
time_dim['month'] = pd.to_datetime(time_dim['date_key']).dt.month
time_dim['quarter'] = pd.to_datetime(time_dim['date_key']).dt.quarter

# Transactions Table (Matches your print(df.columns) showing 'transaction_id')
transactions = df[['transaction_id', 'order_date', 'customer_id', 'product_id', 'final_amount_inr', 'discount_percent', 'is_festival_sale']].copy()
transactions['date_key'] = transactions['order_date'].dt.date
transactions = transactions.drop(columns=['order_date'])

# 4. CREATE DATABASE & TABLES
with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {DATABASE}"))
    conn.execute(text(f"USE {DATABASE}"))
    
    print("🗑️ Resetting tables...")
    conn.execute(text("DROP TABLE IF EXISTS transactions, products, customers, time_dimension"))
    
    conn.execute(text("""
        CREATE TABLE products (
            product_id VARCHAR(100) PRIMARY KEY,
            product_name TEXT,
            category VARCHAR(150),
            brand VARCHAR(150)
        )"""))
    
    conn.execute(text("""
        CREATE TABLE customers (
            customer_id VARCHAR(100) PRIMARY KEY,
            customer_city VARCHAR(150),
            customer_state VARCHAR(150),
            is_prime_member TINYINT(1) DEFAULT 0
        )"""))
    
    conn.execute(text("""
        CREATE TABLE time_dimension (
            date_key DATE PRIMARY KEY,
            year INT, month INT, quarter INT
        )"""))
    
    conn.execute(text("""
        CREATE TABLE transactions (
            transaction_id VARCHAR(100),
            date_key DATE,
            customer_id VARCHAR(100),
            product_id VARCHAR(100),
            final_amount_inr DECIMAL(15,2),
            discount_percent DECIMAL(5,2),
            is_festival_sale TINYINT(1) DEFAULT 0
        )"""))
    conn.commit()

# 5. BULK UPLOAD
print("📤 Uploading to MySQL...")
upload_tasks = [(products, "products"), (customers, "customers"), 
                (time_dim, "time_dimension"), (transactions, "transactions")]

for table_df, name in upload_tasks:
    print(f"   -> {name}...")
    table_df.to_sql(name, engine, if_exists="append", index=False, chunksize=1000)

print("\n🔥 SUCCESS! Check MySQL Workbench for the clean data.")

📥 Reading data...
🧹 Cleaning and Preparing tables...
🗑️ Resetting tables...
📤 Uploading to MySQL...
   -> products...
   -> customers...
   -> time_dimension...
   -> transactions...

🔥 SUCCESS! Check MySQL Workbench for the clean data.
